In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil
print(tf.__version__)

1.11.0


In [7]:
CSV_COLUMNS = ['Fahrenheit', 'Celsius']
LABEL_COLUMN = 'Celsius'
record_defaults = [tf.constant([], dtype=tf.float32),
                   tf.constant([], dtype=tf.float32)]
DEFAULTS = [[0.0], [0.0]]

def read_dataset(filename, mode, batch_size = 512):
  def _input_fn():
    def decode_csv(value_column):
      columns = tf.decode_csv(value_column, record_defaults = DEFAULTS)
      features = dict(zip(CSV_COLUMNS, columns))
      label = features.pop(LABEL_COLUMN)
      return features, label
    
    # Create list of files that match pattern
    file_list = tf.gfile.Glob(filename)

    # Create dataset from file list
    dataset = tf.data.TextLineDataset(file_list).map(decode_csv)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = None # indefinitely
        dataset = dataset.shuffle(buffer_size = 10 * batch_size)
    else:
        num_epochs = 1 # end-of-input after this
 
    dataset = dataset.repeat(num_epochs).batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()
  return _input_fn

In [8]:
feature_cols = [
    tf.feature_column.numeric_column('Fahrenheit')
]

def serving_input_fn():
  feature_placeholders = {
    'Fahrenheit' : tf.placeholder(tf.float32, [None])
  }
  features = {
      key: tf.expand_dims(tensor, -1) for key, tensor in feature_placeholders.items()
  }
  return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)

In [18]:
def train_and_evaluate(output_dir, num_train_steps):
  estimator = tf.estimator.LinearRegressor(
                       model_dir = output_dir,
                       feature_columns = feature_cols)
  train_spec=tf.estimator.TrainSpec(
                       input_fn = read_dataset('./train1.csv', mode = tf.estimator.ModeKeys.TRAIN),
                       max_steps = num_train_steps)
  exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
  eval_spec=tf.estimator.EvalSpec(
                       input_fn = read_dataset('./valid1.csv', mode = tf.estimator.ModeKeys.EVAL),
                       steps = None,
                       start_delay_secs = 1, # start evaluating after N seconds
                       throttle_secs = 10,  # evaluate every N seconds
                       exporters = exporter)
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [ ]:
# Run training    
OUTDIR = './run'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

train_and_evaluate(OUTDIR, num_train_steps = 50000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './run', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11461b710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after ev

INFO:tensorflow:loss = 102.3678, step = 6901 (2.589 sec)
INFO:tensorflow:global_step/sec: 38.834
INFO:tensorflow:loss = 94.10954, step = 7001 (2.575 sec)
INFO:tensorflow:global_step/sec: 31.1458
INFO:tensorflow:loss = 86.520996, step = 7101 (3.211 sec)
INFO:tensorflow:global_step/sec: 37.897
INFO:tensorflow:loss = 79.54382, step = 7201 (2.638 sec)
INFO:tensorflow:global_step/sec: 40.4784
INFO:tensorflow:loss = 73.12903, step = 7301 (2.471 sec)
INFO:tensorflow:global_step/sec: 37.1491
INFO:tensorflow:loss = 67.232254, step = 7401 (2.692 sec)
INFO:tensorflow:global_step/sec: 38.4877
INFO:tensorflow:loss = 61.811203, step = 7501 (2.598 sec)
INFO:tensorflow:global_step/sec: 40.0642
INFO:tensorflow:loss = 56.827328, step = 7601 (2.496 sec)
INFO:tensorflow:global_step/sec: 38.6334
INFO:tensorflow:loss = 52.246727, step = 7701 (2.588 sec)
INFO:tensorflow:global_step/sec: 37.5341
INFO:tensorflow:loss = 48.033684, step = 7801 (2.664 sec)
INFO:tensorflow:global_step/sec: 37.2454
INFO:tensorflow:

INFO:tensorflow:global_step/sec: 33.509
INFO:tensorflow:loss = 0.0963106, step = 15201 (2.984 sec)
INFO:tensorflow:global_step/sec: 32.3473
INFO:tensorflow:loss = 0.088102594, step = 15301 (3.092 sec)
INFO:tensorflow:global_step/sec: 35.974
INFO:tensorflow:loss = 0.081025, step = 15401 (2.780 sec)
INFO:tensorflow:global_step/sec: 31.9869
INFO:tensorflow:loss = 0.07496157, step = 15501 (3.126 sec)
INFO:tensorflow:global_step/sec: 24.5801
INFO:tensorflow:loss = 0.069156796, step = 15601 (4.068 sec)
INFO:tensorflow:global_step/sec: 23.6443
INFO:tensorflow:loss = 0.0635798, step = 15701 (4.230 sec)
INFO:tensorflow:global_step/sec: 24.1058
INFO:tensorflow:loss = 0.058233805, step = 15801 (4.148 sec)
INFO:tensorflow:global_step/sec: 31.2499
INFO:tensorflow:loss = 0.053130068, step = 15901 (3.200 sec)
INFO:tensorflow:global_step/sec: 33.7139
INFO:tensorflow:loss = 0.048893727, step = 16001 (2.966 sec)
INFO:tensorflow:global_step/sec: 33.2059
INFO:tensorflow:loss = 0.04537984, step = 16101 (3.

INFO:tensorflow:global_step/sec: 33.122
INFO:tensorflow:loss = 0.001043946, step = 21501 (3.019 sec)
INFO:tensorflow:global_step/sec: 36.016
INFO:tensorflow:loss = 0.001043946, step = 21601 (2.777 sec)
INFO:tensorflow:global_step/sec: 37.8161
INFO:tensorflow:loss = 0.001043946, step = 21701 (2.646 sec)
INFO:tensorflow:global_step/sec: 38.0346
INFO:tensorflow:loss = 0.001043946, step = 21801 (2.628 sec)
INFO:tensorflow:global_step/sec: 40.4977
INFO:tensorflow:loss = 0.001043946, step = 21901 (2.469 sec)
INFO:tensorflow:global_step/sec: 37.7645
INFO:tensorflow:loss = 0.001043946, step = 22001 (2.648 sec)
INFO:tensorflow:global_step/sec: 38.0486
INFO:tensorflow:loss = 0.001043946, step = 22101 (2.628 sec)
INFO:tensorflow:global_step/sec: 37.7681
INFO:tensorflow:loss = 0.001043946, step = 22201 (2.648 sec)
INFO:tensorflow:global_step/sec: 29.1975
INFO:tensorflow:loss = 0.001043946, step = 22301 (3.425 sec)
INFO:tensorflow:global_step/sec: 33.6903
INFO:tensorflow:loss = 0.001043946, step = 

INFO:tensorflow:global_step/sec: 35.0048
INFO:tensorflow:loss = 0.001043946, step = 29601 (2.857 sec)
INFO:tensorflow:global_step/sec: 38.3078
INFO:tensorflow:loss = 0.001043946, step = 29701 (2.610 sec)
INFO:tensorflow:global_step/sec: 39.7202
INFO:tensorflow:loss = 0.001043946, step = 29801 (2.518 sec)
INFO:tensorflow:global_step/sec: 37.8953
INFO:tensorflow:loss = 0.001043946, step = 29901 (2.639 sec)
INFO:tensorflow:global_step/sec: 39.3235
INFO:tensorflow:loss = 0.001043946, step = 30001 (2.543 sec)
INFO:tensorflow:global_step/sec: 39.7767
INFO:tensorflow:loss = 0.001043946, step = 30101 (2.514 sec)
INFO:tensorflow:global_step/sec: 39.1699
INFO:tensorflow:loss = 0.001043946, step = 30201 (2.553 sec)
INFO:tensorflow:global_step/sec: 39.0431
INFO:tensorflow:loss = 0.001043946, step = 30301 (2.561 sec)
INFO:tensorflow:global_step/sec: 38.2335
INFO:tensorflow:loss = 0.001043946, step = 30401 (2.616 sec)
INFO:tensorflow:global_step/sec: 38.462
INFO:tensorflow:loss = 0.001043946, step =

...

INFO:tensorflow:global_step/sec: 26.5805
INFO:tensorflow:loss = 0.001043946, step = 469101 (3.763 sec)
INFO:tensorflow:global_step/sec: 31.0044
INFO:tensorflow:loss = 0.001043946, step = 469201 (3.224 sec)
INFO:tensorflow:global_step/sec: 35.2736
INFO:tensorflow:loss = 0.001043946, step = 469301 (2.836 sec)
INFO:tensorflow:global_step/sec: 29.2746
INFO:tensorflow:loss = 0.001043946, step = 469401 (3.421 sec)
INFO:tensorflow:global_step/sec: 30.1302
INFO:tensorflow:loss = 0.001043946, step = 469501 (3.314 sec)
INFO:tensorflow:global_step/sec: 28.9914
INFO:tensorflow:loss = 0.001043946, step = 469601 (3.449 sec)
INFO:tensorflow:global_step/sec: 32.1629
INFO:tensorflow:loss = 0.001043946, step = 469701 (3.109 sec)
INFO:tensorflow:global_step/sec: 30.1001
INFO:tensorflow:loss = 0.001043946, step = 469801 (3.322 sec)
INFO:tensorflow:global_step/sec: 29.4958
INFO:tensorflow:loss = 0.001043946, step = 469901 (3.390 sec)
INFO:tensorflow:global_step/sec: 30.071
INFO:tensorflow:loss = 0.00104394

INFO:tensorflow:global_step/sec: 37.2358
INFO:tensorflow:loss = 0.001043946, step = 475301 (2.686 sec)
INFO:tensorflow:global_step/sec: 36.9821
INFO:tensorflow:loss = 0.001043946, step = 475401 (2.705 sec)
INFO:tensorflow:global_step/sec: 38.254
INFO:tensorflow:loss = 0.001043946, step = 475501 (2.613 sec)
INFO:tensorflow:global_step/sec: 38.2672
INFO:tensorflow:loss = 0.001043946, step = 475601 (2.613 sec)
INFO:tensorflow:global_step/sec: 39.1595
INFO:tensorflow:loss = 0.001043946, step = 475701 (2.555 sec)
INFO:tensorflow:global_step/sec: 33.5625
INFO:tensorflow:loss = 0.001043946, step = 475801 (2.978 sec)
INFO:tensorflow:global_step/sec: 39.2381
INFO:tensorflow:loss = 0.001043946, step = 475901 (2.549 sec)
INFO:tensorflow:global_step/sec: 39.1168
INFO:tensorflow:loss = 0.001043946, step = 476001 (2.557 sec)
INFO:tensorflow:global_step/sec: 38.9836
INFO:tensorflow:loss = 0.001043946, step = 476101 (2.564 sec)
INFO:tensorflow:global_step/sec: 38.3925
INFO:tensorflow:loss = 0.00104394

INFO:tensorflow:global_step/sec: 37.0986
INFO:tensorflow:loss = 0.001043946, step = 483301 (2.696 sec)
INFO:tensorflow:global_step/sec: 38.2948
INFO:tensorflow:loss = 0.001043946, step = 483401 (2.611 sec)
INFO:tensorflow:global_step/sec: 32.0394
INFO:tensorflow:loss = 0.001043946, step = 483501 (3.121 sec)
INFO:tensorflow:global_step/sec: 35.662
INFO:tensorflow:loss = 0.001043946, step = 483601 (2.804 sec)
INFO:tensorflow:global_step/sec: 38.7145
INFO:tensorflow:loss = 0.001043946, step = 483701 (2.583 sec)
INFO:tensorflow:global_step/sec: 38.1488
INFO:tensorflow:loss = 0.001043946, step = 483801 (2.622 sec)
INFO:tensorflow:global_step/sec: 30.2382
INFO:tensorflow:loss = 0.001043946, step = 483901 (3.307 sec)
INFO:tensorflow:global_step/sec: 25.1197
INFO:tensorflow:loss = 0.001043946, step = 484001 (3.982 sec)
INFO:tensorflow:global_step/sec: 23.3667
INFO:tensorflow:loss = 0.001043946, step = 484101 (4.279 sec)
INFO:tensorflow:global_step/sec: 27.6564
INFO:tensorflow:loss = 0.00104394

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-08-23:07:30
INFO:tensorflow:Saving dict for global step 490917: average_loss = 2.9665318e-06, global_step = 490917, label/mean = 0.2777772, loss = 1.1866127e-05, prediction/mean = 0.2794404
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 490917: ./run/model.ckpt-490917
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Signatures EXCLUDED from export because they cannot be be served via TensorFlow Serving APIs:
INFO:tensorflow:'serving_default' : Regression input must be a single string 

INFO:tensorflow:global_step/sec: 37.4188
INFO:tensorflow:loss = 0.001043946, step = 497501 (2.673 sec)
INFO:tensorflow:global_step/sec: 37.6214
INFO:tensorflow:loss = 0.001043946, step = 497601 (2.658 sec)
INFO:tensorflow:global_step/sec: 37.5045
INFO:tensorflow:loss = 0.001043946, step = 497701 (2.666 sec)
INFO:tensorflow:global_step/sec: 37.4881
INFO:tensorflow:loss = 0.001043946, step = 497801 (2.668 sec)
INFO:tensorflow:global_step/sec: 37.8984
INFO:tensorflow:loss = 0.001043946, step = 497901 (2.639 sec)
INFO:tensorflow:global_step/sec: 37.9485
INFO:tensorflow:loss = 0.001043946, step = 498001 (2.635 sec)
INFO:tensorflow:global_step/sec: 30.9587
INFO:tensorflow:loss = 0.001043946, step = 498101 (3.230 sec)
INFO:tensorflow:global_step/sec: 36.7836
INFO:tensorflow:loss = 0.001043946, step = 498201 (2.719 sec)
INFO:tensorflow:global_step/sec: 37.1531
INFO:tensorflow:loss = 0.001043946, step = 498301 (2.692 sec)
INFO:tensorflow:global_step/sec: 36.6964
INFO:tensorflow:loss = 0.0010439

In [ ]:
#Predict
OUTDIR = './run'
featcol = [ tf.feature_column.numeric_column('Fahrenheit') ]
model = tf.estimator.LinearRegressor(
      feature_columns = featcol, model_dir = OUTDIR)

def make_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df['Celsius'],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000,
    num_threads = 1
  )

test_data = [[-15,-26.11111111],[5,-15],[30,-1.111111111],[60,15.55555556]]
test_df = pd.DataFrame(test_data,columns=['Fahrenheit','Celsius'])

import itertools
preds_iter = model.predict(input_fn = make_input_fn(test_df, 1))
print([pred['predictions'][0] for pred in list(itertools.islice(preds_iter, 4))])